In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importa as bibliotecas
import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import plot_confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

In [ ]:
#Verifica nulos
df_nan = df.isna().sum()[df.isna().sum() >0]
df_nan

In [ ]:
#Função para imputar valores NA 
def imput_na(df):
    df['v2a1'] = df['v2a1'].fillna(df['v2a1'].median())
    df['v18q1'] = df['v18q1'].fillna(-1)
    df['rez_esc'] = df['rez_esc'].fillna(-1)
    df['meaneduc'] = df['meaneduc'].fillna(df['meaneduc'].median())
    df['SQBmeaned'] = df['SQBmeaned'].fillna(df['SQBmeaned'].median())
    return df

In [ ]:
#Substitui yes e no por 1 e 0

def yes_no(df, column, tipo): 
    map_yes_no = {'yes': 1, 'no': 0}
    df[column] = df[column].replace(map_yes_no).astype(tipo)
    return(df)


In [ ]:
#Imputa
df = imput_na(df)

#Substitui yes e no
df = yes_no(df, 'edjefa', int)
df = yes_no(df, 'edjefe', int)
df = yes_no(df, 'dependency', float)

In [ ]:
#Visualiza o tipo object
df.select_dtypes('object').head()

In [ ]:
# Verificando quantidades e tipos
df.info()

In [ ]:
feats = list(df.select_dtypes(['float','int']).columns)


In [ ]:
## Identificando variáveis

invalida = []
binarios = []
categorica = []
quantitativa = []
for feat in feats:
    if df[feat].nunique() == 1:   
        invalida.append(feat)
    elif df[feat].nunique() == 2:  
        binarios.append(feat)
    elif df[feat].nunique() < 16: 
        categorica.append(feat)
    else:
        quantitativa.append(feat)         

In [ ]:
## Identificando variáveis
invalida = []
binarios = []
categorica = []
quantitativa = []
for feat in feats:
    if df[feat].nunique() == 1:   
        invalida.append(feat)
    elif df[feat].nunique() == 2:  
        binarios.append(feat)
    elif df[feat].nunique() < 16:  
        categorica.append(feat)
    else:
        quantitativa.append(feat)         

In [ ]:
## Histograma de variáveis discretas


ax = df[quantitativa].hist(bins=10, grid=True, figsize=(10,20), rwidth=0.9)
plt.suptitle("Histograma das variáveis discretas", fontsize = 10)
plt.show();

In [ ]:
# Variável Target
#Agrupa por valor
target = (
    pd.DataFrame(df.groupby('Target').size())
    .reset_index()
    .rename(columns={0:'quantidade'})
)
target['Target'] = target.Target.astype(str)
#Plota
plt.figure(figsize=(8,5))
plt.grid(linestyle=':', linewidth=0.7)
plt.bar(target.Target,target.quantidade)
plt.show();

In [ ]:
#Verifica os valores
target['percent'] = round((target.quantidade/target.quantidade.sum())*100)
target

>O desbalanceamento de classe do dataset ocupa 63% da categoria 4 

In [ ]:
#Separa features em target e explicativas
X = df.drop(['Id','Target', 'idhogar','elimbasu5'], axis=1) 
y = df['Target']
#Separa em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Fazendo o over-sampling
ros = RandomOverSampler(random_state=42)
X_ros,y_ros= ros.fit_resample(X_train,y_train)
# Verificando o resultado
y_ros.value_counts()

> # Treinando um modelo de rXGBoost

In [ ]:

xgbc = XGBClassifier(n_estimators=300, learning_rate=0.1, random_state=42)

xgbc.fit(X_ros, y_ros);
# Features
#---------------------------------------------------------------
#Prepara vizualização coeficientes xgbc
xgbc_importance = (
    pd.DataFrame(xgbc.feature_importances_, X_ros.columns)
    .rename(columns={0:'valor'})
    .sort_values('valor')
    .reset_index()  
)

#  Treinando um modelo de AdaBoost


In [ ]:
# Trabalhando com AdaBoost


abc = AdaBoostClassifier(n_estimators=200, learning_rate=1.0, random_state=42)
abc.fit(X_ros, y_ros);
#-----------------------------------------------
abc_importance = (
    pd.DataFrame(abc.feature_importances_, X_ros.columns)
    .rename(columns={0:'valor'})
    .sort_values('valor')
    .reset_index()  
)


# Treinando um modelo de RF Classifier



In [ ]:
# Treinando um modelo de RF Classifier
rfc = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

rfc.fit(X_ros, y_ros);

#Prepara vizualização coeficientes rfc
rfc_importance = (
    pd.DataFrame(rfc.feature_importances_, X_ros.columns)
    .rename(columns={0:'valor'})
    .sort_values('valor')
    .reset_index()  
)



In [ ]:
#Visualiza acurácia
predict_xgbc = xgbc.predict(X_test)
accuracy_xgbc = accuracy_score(y_test, predict_xgbc) * 100
#----------------------------------------------------------
predict_abc = abc.predict(X_test)
accuracy_abc = accuracy_score(y_test, predict_abc) * 100
#--------------------------------------------------------
predict_rfc = rfc.predict(X_test)
accuracy_rfc = accuracy_score(y_test, predict_rfc) * 100
#--------------------------------------------------------
print('Acurácia XGBClassifier:', accuracy_xgbc)
print('Acurácia RFClassifier :', accuracy_rfc)
print('Acurácia AdaBoostClassifier :', accuracy_abc)

In [ ]:

#Matriz de confusão xgbc
plot_confusion_matrix(xgbc,X_test,y_test);

In [ ]:
#Matriz de confusão com dados de teste
plot_confusion_matrix(abc,X_test,y_test);

In [ ]:
#Matriz de confusão com dados de teste
plot_confusion_matrix(rfc,X_test,y_test);

In [ ]:
#Imputa
test = imput_na(test)
##
#Substitui yes e no
test = yes_no(test, 'edjefa', int)
test = yes_no(test, 'edjefe', int)
test = yes_no(test, 'dependency', float)

In [ ]:
#Separa as features explicativas
feats_exp = test.drop(['Id','idhogar','elimbasu5'], axis=1) 

In [ ]:
# Prever o Target de teste usando o modelo com maior acurácia (rfc)
#test['Target'] = rfc.predict(feats_exp).astype(int)

In [ ]:
# Prever o Target de teste usando o modelo xgbc
#test['Target'] = abc.predict(feats_exp).astype(int)


In [ ]:
# Prever o Target de teste usando o modelo com maior acurácia (rfc)
test['Target'] = xgbc.predict(feats_exp).astype(int)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)